# Filtro colaborativo

Por otro lado, tras analizar el feedback de los diseñadores de niveles se ha visto que una de las features más demandas es la recomendación de monstruos a la hora de crear un encuentro, es decir, el grupo que monstruos a los que los jugadores se tiene que enfrentar en una sala. Por ello senecesita implementar un sistema que dada una sala que al menos contenga un monstruo sea capaz  de recomendar otros 5 monstruos que podrían aparecer en el encuentro. Utilizar los datos proporcionados para realizar un filtro colaborativo que recomiende monstruos que se podrían incluir en un encuentro. Impementar en Python dos funciones:


### 1. Una que realice una consulta Cypher y sin usar plugins busque otros monstruos que co- aparezcan en otras salas con los monstruos de nuestro encuentro. El siguiente diagrama explica mejor este concepto.

<p align="center">
    <img src="./recommendations.png" alt="Recommendations" width="500"/>
</p>
<style>
  .centered {
    max-width: 500px;
    margin: 0 auto; /* This centers the element horizontally */
    text-align: center; /* This centers the text inside the element */
  }
</style>
<p class="centered">
  Nos interesa recomendar monstruos para el encuentro de la sala azul, así que buscamos otras salas
donde aparezcan los monstruos de la sala azul, en este caso la sala roja y la verde. A continuación,
listamos aquellos monstruos que aparecen en las salas rojas y verdes, pero no en la azul, estos
monstruos serán los que se recomienden para el encuentro
</p>

In [1]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver('bolt://neo4j:7687', auth=("neo4j", "BDII2023"))
session = driver.session()
def run_query(query, return_data=True, **kwargs):
        results = session.run(query, kwargs)
        if return_data:
            return results.data()
        return results

In [30]:
query = """
// Primero, dado el nombre de la habitación, encuentra los monstruos en esa habitación.
MATCH (r:Room{room_name: $room})-[c:CONTAINS]->(m:Monster)
WITH r, m
// Luego, encuentra todas las otras habitaciones que contengan al menos uno de esos monstruos.
MATCH (r2:Room)-[c:CONTAINS]->(m)
// Asegúrate de que no estamos mirando la misma habitación.
WHERE r2 <> r
WITH r2 //, m // Si m2 <> m no es usado, esto no se necesita.
// Finalmente, cuenta cuántas veces aparece cada monstruo en las otras habitaciones.
MATCH (r2)-[c:CONTAINS]->(m2:Monster)
// WHERE m2 <> m // Esto no funciona
// Dado que queremos recomendar monstruos que no estén ya en la habitación, filtramos los monstruos que ya están en la habitación utilizando lo siguiente:
WHERE NOT EXISTS {MATCH (Room{room_name: $room})-[:CONTAINS]->(m2)}
RETURN m2.name AS name, COUNT(m2) as count
ORDER BY count DESC
"""
room = "sexy garden of presidents"
results = run_query(query, room=room)
results

[{'name': 'doppelganger', 'count': 122},
 {'name': 'revenant', 'count': 98},
 {'name': 'vampire spawn', 'count': 94},
 {'name': 'thug', 'count': 32},
 {'name': 'assassin', 'count': 20}]

In [40]:
def recommend_monsters(room):
    query = """
    // Primero, dado el nombre de la habitación, encuentra los monstruos en esa habitación.
    MATCH (r:Room{room_name: $room})-[c:CONTAINS]->(m:Monster)
    WITH r, m
    // Luego, encuentra todas las otras habitaciones que contengan al menos uno de esos monstruos.
    MATCH (r2:Room)-[c:CONTAINS]->(m)
    // Asegúrate de que no estamos mirando la misma habitación.
    WHERE r2 <> r
    WITH r2 //, m // Si m2 <> m no es usado, esto no se necesita.
    // Finalmente, cuenta cuántas veces aparece cada monstruo en las otras habitaciones.
    MATCH (r2)-[c:CONTAINS]->(m2:Monster)
    // WHERE m2 <> m // Esto no funciona
    // Dado que queremos recomendar monstruos que no estén ya en la habitación, filtramos los monstruos que ya están en la habitación utilizando lo siguiente:
    WHERE NOT EXISTS {MATCH (Room{room_name: $room})-[:CONTAINS]->(m2)}
    RETURN m2.name AS name, COUNT(m2) as count
    ORDER BY count DESC
    """
    results = run_query(query, room=room)
    return results

In [41]:
recommend_monsters("sexy garden of presidents")

[{'name': 'doppelganger', 'count': 122},
 {'name': 'revenant', 'count': 98},
 {'name': 'vampire spawn', 'count': 94},
 {'name': 'thug', 'count': 32},
 {'name': 'assassin', 'count': 20}]

### 2. Una que utilice el plugin de GDS para realizar una recomendación de 5 monstruos ordenados del que mas recomendaría al que menos recomendaría

In [3]:
from graphdatascience import GraphDataScience

gds = GraphDataScience("neo4j://neo4j:7687", auth=("neo4j", "BDII2023"))

Primero, creamos una proyección de los nodos de monstruos y salas, y las relaciones entre ellos. La dirección de la relación se hace no dirigida para que el PageRank personalizado funcione.

In [15]:
gds.graph.drop("rooms_monsters")

In [16]:
try:
    gds.graph.drop("rooms_monsters")
except:
    pass

G, result = gds.graph.project(
  'rooms_monsters',
  ['Room', 'Monster'],
  {'CONTAINS': {'orientation': 'UNDIRECTED'}}
)

Para usar el PageRank personalizado, se necesita un nodo de inicio. Para ello, como el algoritmo toma como entrada el id del nodo, usamos la función `gds.find_node_id()` para encontrar el id de la habitacíon en cuetión.

In [5]:
source_id = gds.find_node_id(["Room"], {"room_name": "sexy garden of presidents"})
source_id

5748

Ejecutamos el algoritmo, en stream, por lo que no guardamos los resultados en la base de datos. En su lugar, nos devuelve un DataFrame con los resultados.

In [24]:
pagerank = gds.pageRank.stream(G, maxIterations=20, dampingFactor=0.85, sourceNodes=[source_id]).sort_values("score", ascending=False)
pagerank

,nodeId,score
5748,5748,0.151192
11213,11213,0.144845
11212,11212,0.144378
11211,11211,0.049783
11215,11215,0.039955
...,...,...
4005,4005,0.000000
4006,4006,0.000000
4007,4007,0.000000
4008,4008,0.000000


Vemos que nos da los ids de los nodos, por lo que usamos la función `gds.util.asNodes()` para convertirlos en nodos de la base de datos.

In [25]:
pagerank['node'] = gds.util.asNodes(pagerank['nodeId'].to_list())
pagerank

,nodeId,score,node
5748,5748,0.151192,"(room_id, room_name, componentId, dungeon_name)"
11213,11213,0.144845,"(level, name, place, exp, type, monsters_id)"
11212,11212,0.144378,"(level, name, place, exp, type, monsters_id)"
11211,11211,0.049783,"(level, name, place, exp, type, monsters_id)"
11215,11215,0.039955,"(level, name, place, exp, type, monsters_id)"
...,...,...,...
4005,4005,0.000000,"(room_id, room_name, componentId, dungeon_name)"
4006,4006,0.000000,"(room_id, room_name, componentId, dungeon_name)"
4007,4007,0.000000,"(room_id, room_name, componentId, dungeon_name)"
4008,4008,0.000000,"(room_id, room_name, componentId, dungeon_name)"


Vemos que el algoritmo nos devuelve nodos de tipo `Monster` y de tipo `Room`. Primero, creamos una nueva columna en el DataFrame, con el que nos quedamos con la propiedad `name` de todos los nodos

In [26]:
pagerank['monster_name'] = pagerank['node'].apply(lambda x: x['name'])

In [27]:
pagerank

,nodeId,score,node,monster_name
5748,5748,0.151192,"(room_id, room_name, componentId, dungeon_name)",None
11213,11213,0.144845,"(level, name, place, exp, type, monsters_id)",wererat
11212,11212,0.144378,"(level, name, place, exp, type, monsters_id)",gargoyle
11211,11211,0.049783,"(level, name, place, exp, type, monsters_id)",doppelganger
11215,11215,0.039955,"(level, name, place, exp, type, monsters_id)",revenant
...,...,...,...,...
4005,4005,0.000000,"(room_id, room_name, componentId, dungeon_name)",None
4006,4006,0.000000,"(room_id, room_name, componentId, dungeon_name)",None
4007,4007,0.000000,"(room_id, room_name, componentId, dungeon_name)",None
4008,4008,0.000000,"(room_id, room_name, componentId, dungeon_name)",None


Vemos que los nodos `Room` no tienen la propiedad `name`, por lo que tienen un valor `None`. Por ello, hacemos un `dropna()` para eliminar estos nodos, queándonos solo con los nodos `Monster`.

In [28]:
pagerank[['monster_name', 'score']].dropna().head(10)

,monster_name,score
11213,wererat,0.144845
11212,gargoyle,0.144378
11211,doppelganger,0.049783
11215,revenant,0.039955
11217,vampire spawn,0.037234
11224,thug,0.016254
11220,assassin,0.009202
11468,piercer,0.000000
11469,quaggoth,0.000000
11470,roper,0.000000


Vemos que los monstruos que nos devuelve el algoritmo con un _score_ mayor que 0 son los mismos monstruos que aparecen en la primera query, incluyendo también los monstrus de la propia sala. Para quedarnos solo con los monstruos que no aparecen en la sala, obtenemos los ids de los monstruos de la sala y los eliminamos del DataFrame.

In [34]:
query = """
MATCH (r:Room{room_name: $room})-[c:CONTAINS]->(m:Monster)
// return the id of the monster
RETURN id(m) as monster_id"""
monsters_in_room = gds.run_cypher(query, {"room": "sexy garden of presidents"})
monsters_in_room

,monster_id
0,11212
1,11213


In [35]:
pagerank = pagerank[~pagerank['nodeId'].isin(monsters_in_room['monster_id'])]

In [37]:
pagerank[['monster_name', 'score']].dropna().head(5)

,monster_name,score
11211,doppelganger,0.049783
11215,revenant,0.039955
11217,vampire spawn,0.037234
11224,thug,0.016254
11220,assassin,0.009202


In [42]:
def recommend_monsters_gds(gds, room):
    try:
        gds.graph.drop("rooms_monsters")
    except:
        pass

    G, result = gds.graph.project(
    'rooms_monsters',
    ['Room', 'Monster'],
    {'CONTAINS': {'orientation': 'UNDIRECTED'}}
    )
    source_id = gds.find_node_id(["Room"], {"room_name": room})
    pagerank = gds.pageRank.stream(G, maxIterations=20, dampingFactor=0.85, sourceNodes=[source_id]).sort_values("score", ascending=False)
    pagerank['node'] = gds.util.asNodes(pagerank['nodeId'].to_list())
    pagerank['monster_name'] = pagerank['node'].apply(lambda x: x['name'])
    monsters_in_room = gds.run_cypher(query, {"room": room})
    pagerank = pagerank[~pagerank['nodeId'].isin(monsters_in_room['monster_id'])]
    return pagerank[['monster_name', 'score']].dropna().head(5)

In [43]:
recommend_monsters_gds(gds, "sexy garden of presidents")

,monster_name,score
11211,doppelganger,0.049783
11215,revenant,0.039955
11217,vampire spawn,0.037234
11224,thug,0.016254
11220,assassin,0.009202
